In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw""
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
using Arrow, MixedModels, StatsModels

In [3]:
CYC_2022 = Arrow.Table("data.arrow")

Arrow.Table with 1413 rows, 6 columns, and schema:
 :Accuracy      Union{Missing, Float64}
 :Condition     Union{Missing, String}
 :SemanticFit   Union{Missing, Float64}
 :Transitivity  Union{Missing, Float64}
 :Item          Union{Missing, String}
 :Subject       Union{Missing, Int32}

In [4]:
fm = @formula(
  Accuracy ~ Condition * SemanticFit * Transitivity +
             (1 + Condition * SemanticFit * Transitivity | Subject) +
             (1 + Condition | Item)
)

FormulaTerm
Response:
  Accuracy(unknown)
Predictors:
  Condition(unknown)
  SemanticFit(unknown)
  Transitivity(unknown)
  (Condition,SemanticFit,Transitivity,Subject)->(1 + Condition * SemanticFit * Transitivity) | Subject
  (Condition,Item)->(1 + Condition) | Item
  Condition(unknown) & SemanticFit(unknown)
  Condition(unknown) & Transitivity(unknown)
  SemanticFit(unknown) & Transitivity(unknown)
  Condition(unknown) & SemanticFit(unknown) & Transitivity(unknown)

In [5]:
coding = Dict(
  :Subject => Grouping(),
  :Item => Grouping(),
  :Condition => EffectsCoding(base = "Subject")
)

Dict{Symbol, AbstractContrasts} with 3 entries:
  :Item      => Grouping()
  :Subject   => Grouping()
  :Condition => EffectsCoding("Subject", nothing)

In [6]:
#| code-fold: true
m_maximal = @time fit(MixedModel, fm, CYC_2022, Binomial(), contrasts = coding)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\jchoe\.julia\packages\ProgressMeter\sN2xr\src\ProgressMeter.jl:618
Minimizing 3403 	 Time: 0:00:34 (10.10 ms/it)
  objective:  1217.8703476613064


 81.111613 seconds (95.95 M allocations: 5.827 GiB, 2.22% gc time, 59.13% compilation time: 6% of which was recompilation)


|                                              |    Est. |     SE |     z |      p | σ_Subject | σ_Item |
|:-------------------------------------------- | -------:| ------:| -----:| ------:| ---------:| ------:|
| (Intercept)                                  |  1.8434 | 0.1952 |  9.45 | <1e-20 |    0.9930 | 0.5580 |
| Condition: Verb                              | -0.2287 | 0.0887 | -2.58 | 0.0099 |    0.2914 | 0.0007 |
| SemanticFit                                  | -0.5677 | 0.1413 | -4.02 | <1e-04 |    0.1290 |        |
| Transitivity                                 | -0.4029 | 0.1601 | -2.52 | 0.0119 |    0.0789 |        |
| Condition: Verb & SemanticFit                | -0.0642 | 0.0826 | -0.78 | 0.4372 |    0.1502 |        |
| Condition: Verb & Transitivity               | -0.1180 | 0.0943 | -1.25 | 0.2108 |    0.2206 |        |
| SemanticFit & Transitivity                   | -0.5050 | 0.1758 | -2.87 | 0.0041 |    0.4297 |        |
| Condition: Verb & SemanticFit & Transitivity |  0.0091 | 0.0981 |  0.09 | 0.9259 |    0.0538 |        |


In [7]:
issingular(m_maximal)

true

In [8]:
fm_zerocorr = @formula(
  Accuracy ~ Condition * SemanticFit * Transitivity +
             zerocorr(1 + Condition * SemanticFit * Transitivity | Subject) +
             zerocorr(1 + Condition | Item)
)

FormulaTerm
Response:
  Accuracy(unknown)
Predictors:
  Condition(unknown)
  SemanticFit(unknown)
  Transitivity(unknown)
  (Condition,SemanticFit,Transitivity,Subject)->zerocorr((1 + Condition * SemanticFit * Transitivity) | Subject)
  (Condition,Item)->zerocorr((1 + Condition) | Item)
  Condition(unknown) & SemanticFit(unknown)
  Condition(unknown) & Transitivity(unknown)
  SemanticFit(unknown) & Transitivity(unknown)
  Condition(unknown) & SemanticFit(unknown) & Transitivity(unknown)

In [9]:
#| code-fold: true
m_maximal_zerocorr = @time fit(MixedModel, fm_zerocorr, CYC_2022, Binomial(), contrasts = coding)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\jchoe\.julia\packages\ProgressMeter\sN2xr\src\ProgressMeter.jl:618
Minimizing 608 	 Time: 0:00:04 ( 8.02 ms/it)
  objective:  1227.8122818506772


  7.242609 seconds (3.27 M allocations: 181.544 MiB, 0.92% gc time, 32.40% compilation time)


|                                              |    Est. |     SE |     z |      p | σ_Subject | σ_Item |
|:-------------------------------------------- | -------:| ------:| -----:| ------:| ---------:| ------:|
| (Intercept)                                  |  1.7418 | 0.1872 |  9.31 | <1e-19 |    0.9366 | 0.5456 |
| Condition: Verb                              | -0.1796 | 0.0823 | -2.18 | 0.0292 |    0.1985 | 0.0000 |
| SemanticFit                                  | -0.4906 | 0.1367 | -3.59 | 0.0003 |    0.0000 |        |
| Transitivity                                 | -0.3679 | 0.1565 | -2.35 | 0.0187 |    0.0000 |        |
| Condition: Verb & SemanticFit                | -0.0983 | 0.0784 | -1.25 | 0.2098 |    0.0000 |        |
| Condition: Verb & Transitivity               | -0.0109 | 0.0876 | -0.12 | 0.9009 |    0.0165 |        |
| SemanticFit & Transitivity                   | -0.4521 | 0.1665 | -2.72 | 0.0066 |    0.2974 |        |
| Condition: Verb & SemanticFit & Transitivity |  0.0080 | 0.0959 |  0.08 | 0.9336 |    0.0000 |        |


In [10]:
issingular(m_maximal_zerocorr)

true

In [11]:
m_maximal_zerocorr.rePCA

(Subject = [0.25, 0.5, 0.75, 1.0, 1.0, 1.0, 1.0, 1.0], Item = [1.0, 1.0])

In [12]:
fm_zerocorr_reduced = @formula(
  Accuracy ~ Condition * SemanticFit * Transitivity +
             zerocorr(1 + Condition | Subject) +
             (1 | Item)
)

FormulaTerm
Response:
  Accuracy(unknown)
Predictors:
  Condition(unknown)
  SemanticFit(unknown)
  Transitivity(unknown)
  (Condition,Subject)->zerocorr((1 + Condition) | Subject)
  (Item)->1 | Item
  Condition(unknown) & SemanticFit(unknown)
  Condition(unknown) & Transitivity(unknown)
  SemanticFit(unknown) & Transitivity(unknown)
  Condition(unknown) & SemanticFit(unknown) & Transitivity(unknown)

In [13]:
#| code-fold: true
m_maximal_zerocorr_reduced = @time fit(MixedModel, fm_zerocorr_reduced, CYC_2022, Binomial(), contrasts = coding)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\jchoe\.julia\packages\ProgressMeter\sN2xr\src\ProgressMeter.jl:618
Minimizing 905 	 Time: 0:00:02 ( 3.10 ms/it)


  5.428734 seconds (2.96 M allocations: 153.321 MiB, 47.78% compilation time)


|                                              |    Est. |     SE |     z |      p | σ_Subject | σ_Item |
|:-------------------------------------------- | -------:| ------:| -----:| ------:| ---------:| ------:|
| (Intercept)                                  |  1.7099 | 0.1846 |  9.26 | <1e-19 |    0.9293 | 0.5331 |
| Condition: Verb                              | -0.1794 | 0.0815 | -2.20 | 0.0277 |    0.1869 |        |
| SemanticFit                                  | -0.4684 | 0.1342 | -3.49 | 0.0005 |           |        |
| Transitivity                                 | -0.3672 | 0.1538 | -2.39 | 0.0170 |           |        |
| Condition: Verb & SemanticFit                | -0.1012 | 0.0772 | -1.31 | 0.1895 |           |        |
| Condition: Verb & Transitivity               | -0.0099 | 0.0871 | -0.11 | 0.9098 |           |        |
| SemanticFit & Transitivity                   | -0.4362 | 0.1581 | -2.76 | 0.0058 |           |        |
| Condition: Verb & SemanticFit & Transitivity |  0.0187 | 0.0948 |  0.20 | 0.8440 |           |        |


In [14]:
issingular(m_maximal_zerocorr_reduced)

false

In [15]:
fm_corr_reduced = @formula(
  Accuracy ~ Condition * SemanticFit * Transitivity +
             (1 + Condition | Subject) +
             (1 | Item)
)

FormulaTerm
Response:
  Accuracy(unknown)
Predictors:
  Condition(unknown)
  SemanticFit(unknown)
  Transitivity(unknown)
  (Condition,Subject)->(1 + Condition) | Subject
  (Item)->1 | Item
  Condition(unknown) & SemanticFit(unknown)
  Condition(unknown) & Transitivity(unknown)
  SemanticFit(unknown) & Transitivity(unknown)
  Condition(unknown) & SemanticFit(unknown) & Transitivity(unknown)

In [16]:
#| code-fold: true
m_maximal_corr_reduced = @time fit(MixedModel, fm_corr_reduced, CYC_2022, Binomial(), contrasts = coding)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\jchoe\.julia\packages\ProgressMeter\sN2xr\src\ProgressMeter.jl:618
Minimizing 1080 	 Time: 0:00:03 ( 3.40 ms/it)


  4.457088 seconds (1.58 M allocations: 77.417 MiB, 17.24% compilation time)


|                                              |    Est. |     SE |     z |      p | σ_Subject | σ_Item |
|:-------------------------------------------- | -------:| ------:| -----:| ------:| ---------:| ------:|
| (Intercept)                                  |  1.7294 | 0.1874 |  9.23 | <1e-19 |    0.9545 | 0.5347 |
| Condition: Verb                              | -0.2372 | 0.0843 | -2.81 | 0.0049 |    0.2355 |        |
| SemanticFit                                  | -0.4688 | 0.1345 | -3.49 | 0.0005 |           |        |
| Transitivity                                 | -0.3688 | 0.1543 | -2.39 | 0.0168 |           |        |
| Condition: Verb & SemanticFit                | -0.0877 | 0.0773 | -1.14 | 0.2563 |           |        |
| Condition: Verb & Transitivity               |  0.0007 | 0.0874 |  0.01 | 0.9940 |           |        |
| SemanticFit & Transitivity                   | -0.4400 | 0.1584 | -2.78 | 0.0055 |           |        |
| Condition: Verb & SemanticFit & Transitivity |  0.0293 | 0.0949 |  0.31 | 0.7578 |           |        |


In [17]:
issingular(m_maximal_corr_reduced)

false